In [1]:
import numpy as np
import pandas as pd

### Load the data

In [2]:
df_train_feature = pd.read_csv('../data.nosync/raw/train_features.csv')
df_train_target = pd.read_csv('../data.nosync/raw/train_salaries.csv')
df_test_feature = pd.read_csv('../data.nosync/raw/test_features.csv')

### Merge data and clean up

Merge training feature to target data and as per EDA do the following:

Reanme the columns.

Get rid of the outliers as per EDA. 

Since the missing values are imputed as "NONE" we do not need to take care of them.

Finally before we encode the categorical data, we shffle the data to make sure there is no bias in train\test split.

In [3]:
df_train = df_train_feature.merge(df_train_target, on='jobId')
df_train.drop(["jobId"], axis=1, inplace=True)

***Rename columns***

In [4]:
def rename_cols(df_list, col_list, new_col_list):

    name_map = dict(zip(col_list, new_col_list))
    
    for df in df_list:
        if any([elem for elem in col_list if elem in df.columns]):
            this_map = dict((k, v) for k, v in name_map.items() if k in df.columns)
            df.rename(columns=this_map, inplace=True)
    return

rename_cols([df_train, df_test_feature], ['jobType'], ['jobLevel'])

***Merge and Clean outliers***

In [5]:
def get_iqr_bounds(df, feature, alpha = 1.5):
    
    if not isinstance(feature, str):
        raise TypeError('feature should be string, indicating the name of the feature')
    
    stats = df[feature].describe()
    iqr = stats['75%'] - stats['25%']
    lower, upper = stats['25%'] - alpha * iqr, stats['75%'] + alpha * iqr
    return lower, upper

salary_lower_bound, salary_upper_bound = get_iqr_bounds(df_train, 'salary')
df_train = df_train[df_train.salary > salary_lower_bound]

***Shuffle and re-index***

In [6]:
# df_train.sample(frac=1).reset_index(inplace=True, drop=True)
# df_train.head()

In [7]:
df_train.groupby("degree")["salary"].mean()

degree
BACHELORS      125.454663
DOCTORAL       135.490979
HIGH_SCHOOL    101.921085
MASTERS        130.505647
NONE            98.176882
Name: salary, dtype: float64

In [8]:
df_train.groupby("degree")["salary"].std()

degree
BACHELORS      35.042720
DOCTORAL       35.790555
HIGH_SCHOOL    36.112684
MASTERS        35.399830
NONE           34.915141
Name: salary, dtype: float64

### Encode Categorical features

For the categorical features we are going to create three type of encoding. We will examine which of these features will work better with our baseline method.

The three encoding methods are:

1) **Ordinal - proportional**: in this method we will replace each cateorical values with the mean of the target value for that category. This guarantees that each category gets a weight proportional to it's values. However, since these values are calculated from test daya only 
    
2) **Ordinal - sorted**: in this method we will replace each categorical values with an integer, but the order of integers are the same as the order of the mean of the target value for that category. This will eliminate the bias that proportional feature might have. 

3) **OneHot**: this is the traditional one-hot encoding.

***Categorical feature encoder classes***

In [9]:
import enum
class OrdinalEncodeType(enum.Enum):
    
    _sorted = 1
    _proportional = 2

In [10]:
class OrdinalEncoder():
    
    def __init__(self, df, cat_features_list, target_feature, encode_type=OrdinalEncodeType._sorted, drop=True):
        if any([elem not in df.columns for elem in cat_features_list+[target_feature]]):
            raise ValueError("one of the features_list or target_feature is not in the DataFrame")
        
        self.df = df
        self.cat_features = cat_features_list
        self.target_feature = target_feature
        self.encoder_method, self.encoded_prefix = self._get_encoder_methd_and_suffix(encode_type)
        self.is_drop = drop
        self.is_encoded = False
    
    def get_encoded_features(self):
        if not self.is_encoded:
            self.encode()
        return self.df
            
    def encode(self):
        for feature in self.cat_features:
            encoder_map = self.encoder_method(feature, self.target_feature).to_dict()
            self.df[self.encoded_prefix + feature] = self.df[feature].replace(encoder_map)
            
        if self.is_drop:
            self.df.drop(columns=self.cat_features)
        
        self.is_encoded = True
        return None
    
    def _get_encoder_methd_and_suffix(self, encode_type):
        if not isinstance(encode_type, OrdinalEncodeType):
            raise valueError(f"'encode_type' shold be an instance of OrdinalEncodeType, whereas it is {type(how)}")
        
        return {
            OrdinalEncodeType._sorted: (self._sorted_encoder, "ORDINAL_SORTED_"),
            OrdinalEncodeType._proportional: (self._proportional_encoder, "ORDINAL_PROPORTIONAL_")
        }[encode_type]
    
    def _proportional_encoder(self, feature, target_feature):
        return self.df.groupby(feature)[self.target_feature].mean().sort_values()
    
    def _sorted_encoder(self, feature, target_feature):
        new_df = self._proportional_encoder(feature, target_feature)
        return pd.Series(list(range(new_df.size)), index = new_df.index)

In [19]:
class OneHotEncoder():
    
    def __init__(self, df, cat_features_list, drop=True):
        if any([elem not in df.columns for elem in cat_features_list]):
            raise ValueError("one of the features_list or target_feature is not in the DataFrame")
        
        self.df = df
        self.cat_features = cat_features_list
        self.is_drop = drop
        self.prefix = "1HOT_"
        self.is_encoded = False
        
    def encode(self):
        for feature in self.cat_features:
            one_hot = pd.get_dummies(self.df[feature], prefix=self.prefix+feature)
            self.df = pd.concat([self.df, one_hot], axis=1)
        
        if self.is_drop:
            self.df.drop(columns=self.cat_features, inplace=True)
        
        self.is_encoded = True
        return None
    
    def get_encoded_features(self):
        if not self.is_encoded:
            self.encode()
        return self.df

***Encode categorical features in the trainig data***

In [20]:
# Create Ordinal-Proportional encdoded data
ord_prop_encoder = OrdinalEncoder(df_train[["jobLevel", "degree", "major", "industry", "salary"]], ["jobLevel", "degree", "major", "industry"], "salary", encode_type=OrdinalEncodeType._proportional)
ord_prop_data = ord_prop_encoder.get_encoded_features()

# Create Ordinal-Sorted encdoded data
ord_sorted_encoder = OrdinalEncoder(df_train[["jobLevel", "degree", "major", "industry", "salary"]], ["jobLevel", "degree", "major", "industry"], "salary", encode_type=OrdinalEncodeType._sorted)
ord_sorted_data = ord_sorted_encoder.get_encoded_features()

# Create One-Hot encdoded data
one_hot_encoder = OneHotEncoder(df_train[["jobLevel", "degree", "major", "industry"]], ["jobLevel", "degree", "major", "industry"])
one_hot_data = one_hot_encoder.get_encoded_features()


/Users/rezakabiri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
one_hot_data

,jobLevel,degree,major,industry,1HOT_jobLevel_CEO,1HOT_jobLevel_CFO,1HOT_jobLevel_CTO,1HOT_jobLevel_JANITOR,1HOT_jobLevel_JUNIOR,1HOT_jobLevel_MANAGER,...,1HOT_major_MATH,1HOT_major_NONE,1HOT_major_PHYSICS,1HOT_industry_AUTO,1HOT_industry_EDUCATION,1HOT_industry_FINANCE,1HOT_industry_HEALTH,1HOT_industry_OIL,1HOT_industry_SERVICE,1HOT_industry_WEB
0,CFO,MASTERS,MATH,HEALTH,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,CEO,HIGH_SCHOOL,NONE,WEB,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,MANAGER,DOCTORAL,CHEMISTRY,AUTO,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
999996,CTO,HIGH_SCHOOL,NONE,FINANCE,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
999997,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
999998,CFO,MASTERS,NONE,HEALTH,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [14]:
# Placeholder for categorical combination for 
# three most important features - (1-HOT)

### Save data to csv

In [15]:
df_train.to_csv('../data.nosync/processed/train_data.csv')
ord_prop_data.to_csv('../data.nosync/processed/ord_prop_data.csv')
ord_sorted_data.to_csv('../data.nosync/processed/ord_sorted_data.csv')
one_hot_data.to_csv('../data.nosync/processed/one_hot_data.csv')
df_train['Salary'].to_csv('../data.nosync/processed/salary_data.csv')

In [16]:
df_train.head()

,companyId,jobLevel,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163
